# Numerical PDE III: Spectral-Galerkin Method

In fluid dynamics, spectral-Galerkin methods are commonly used to
solve the incompressible Navier-Stokes equations.
By transforming the equations into spectral space, the nonlinear terms
can be computed efficiently while maintaining conservation properties;
and the viscous term can be easily solve implicitly.
For incompressible flows, the vorticity-streamfunction formulation is
particularly usable, as it eliminates the pressure term and reduces
the computational complexity.

In this lecture, we will focus on applying spectral methods to solve
the 2D incompressible hydrodynamics equations.
Specifically, we will:
1. Derive the governing equations in spectral space.
2. Discuss the vorticity-streamfunction formulation and its
   advantages.
3. Implement a spectral solver to simulate the evolution of vorticity
   in a 2D periodic domain.

This introduction sets the stage for understanding how spectral
methods leverage mathematical elegance and computational efficiency to
solve complex PDEs with remarkable accuracy.

## 2D Incompressible Hydrodynamics

The hydrodynamic equations govern the conservation of mass and
momentum in a fluid.

In their compressible form (that we derived in previous lectures),
they are written as:
\begin{align}
  \frac{\partial \rho}{\partial t} + \nabla \cdot (\rho \mathbf{u})
  &= 0,
  \quad \text{(Continuity Equation)} \\
  \frac{\partial (\rho \mathbf{u})}{\partial t} + \nabla \cdot (\rho \mathbf{u} \mathbf{u})
  &= -\nabla P + \mu \nabla^2 \mathbf{u} + \mathbf{F},
  \quad \text{(Momentum Equation)}
\end{align}
where:
* $\rho$ is the density,
* $\mathbf{u}$ is the velocity field,
* $P$ is the pressure,
* $\mu$ is the dynamic viscosity, and
* $\mathbf{F}$ is an external force.

In the incompressible limit, the sound speed approaches infinite $c
\rightarrow \infty$.
For simplicity, the density $\rho$ can be assumed constant, and the
continuity equation reduces to the incompressibility condition:
\begin{align}
  \nabla \cdot \mathbf{u} = 0.
\end{align}

Substituting this condition into the momentum equation simplifies the
Navier-Stokes equations to:
\begin{align}
  \frac{\partial \mathbf{u}}{\partial t} + (\mathbf{u} \cdot \nabla) \mathbf{u}
  &= -\nabla p + \nu \nabla^2 \mathbf{u} + \mathbf{f},
\end{align}
where $\nu \equiv \mu / \rho$ is the kinematic viscosity and
$\mathbf{f} \equiv \mathbf{F} / \rho$ is usually still called external
"force".
These equations describe the flow of incompressible fluids.
They are widely used in modeling low Mach number fluid from
small-scale laboratory experiments to large-scale geophysical flows.

While the incompressible Navier-Stokes equations describes
three-dimensional flows, many physical systems can be effectively
approximated as two-dimensional.
For example:
* Atmospheric flows and ocean currents are largely horizontal due to
  their vast spatial extent compared to their depth.
* Thin liquid films and confined flows are geometrically restricted to
  two dimensions.

To simplify the mathematical and computational treatment of 2D
incompressible hydrodynamics, the governing equations are often
reformulated in terms of the vorticity $w$ and the streamfunction
$\psi$.

This formulation has several advantages: it eliminates the pressure
term from the equations, reduces the number of variables, and ensures
incompressibility is automatically satisfied.

### Vorticity-Streamfunction Formulation

For 2D incompressible flows, the velocity field
$\mathbf{u} = (u_x, u_y)$ can be expressed in terms of a scalar function,
the streamfunction $\psi(x, y, t)$, as:
\begin{align}
  \mathbf{u} = \nabla \times (\psi \mathbf{\hat{z}}),
\end{align}
where $\mathbf{\hat{z}}$ is the unit vector perpendicular to the 2D plane.
In component form:
\begin{align}
  u_x = \frac{\partial \psi}{\partial y}, \quad u_y = -\frac{\partial \psi}{\partial x}.
\end{align}

This representation automatically satisfies the incompressibility
condition:
\begin{align}
  \nabla \cdot \mathbf{u}
  = \frac{\partial u_x}{\partial x} + \frac{\partial u_y}{\partial y} = 0.
\end{align}

The vorticity $\omega$ is a scalar quantity in 2D, defined as the curl
of the velocity field:
\begin{align}
  \omega = \nabla \times \mathbf{u}.
\end{align}

Using the velocity components in terms of the streamfunction, the
vorticity can be written as:
\begin{align}
  \omega = \frac{\partial u_y}{\partial x} - \frac{\partial u_x}{\partial y} = -\nabla^2 \psi.
\end{align}

Thus, the vorticity and streamfunction are related by the Poisson
equation:
\begin{align}
  \omega = -\nabla^2 \psi.
\end{align}
This relationship allows the velocity field to be computed from the
vorticity by solving the Poisson equation for $\psi$, and then taking
derivatives of $\psi$ to find $u_x$ and $u_y$.

The vorticity transport equation is derived from the incompressible
Navier-Stokes equations.
Taking the curl of the momentum equation eliminates the pressure
gradient term,
\begin{align}
  \frac{\partial\omega}{\partial t} + \mathbf{u} \cdot \nabla\omega = \nu \nabla^2\omega + \mathbf{f}_\omega,
\end{align}
where:
* $\mathbf{u} \cdot \nabla\omega$ represents the nonlinear advection
  of vorticity,
* $\nu \nabla^2\omega$ accounts for viscous diffusion,
* $\mathbf{f}_\omega$ is the vorticity-specific forcing term.

The term $\mathbf{u} \cdot \nabla\omega$ can be expanded using the
velocity components as:
\begin{align}
  \mathbf{u} \cdot \nabla\omega = u_x \frac{\partial\omega}{\partial x} + u_y \frac{\partial\omega}{\partial y}.
\end{align}

By substituting $u_x$ and $u_y$ in terms of $\psi$, the nonlinear
advection term is rewritten as the Jacobian determinant:
\begin{align}
  J(\psi, \omega) = \frac{\partial \psi}{\partial x} \frac{\partial\omega}{\partial y} - \frac{\partial \psi}{\partial y} \frac{\partial\omega}{\partial x}.
\end{align}
Thus, the vorticity transport equation becomes:
\begin{align}
  \frac{\partial\omega}{\partial t} - J(\psi, \omega) = \nu \nabla^2\omega + \mathbf{f}_\omega.
\end{align}


### Incorporating Additional Physical Effects

Ekman damping models frictional effects caused by the interaction of
the fluid with a boundary layer.
It acts as a large-scale energy sink and is proportional to the
vorticity:
\begin{align}
  -\mu\omega,
\end{align}
where $\mu$ is the Ekman coefficient.

Including this term in the vorticity transport equation gives:
\begin{align}
  \frac{\partial\omega}{\partial t} - J(\psi, \omega)
  = \nu \nabla^2\omega - \mu\omega + \mathbf{f}_\omega.
\end{align}

Ekman damping is particularly relevant in geophysical systems, where
it represents energy dissipation due to the Earth's surface or ocean
floors.

The $\beta$-plane approximation models the variation of the Coriolis
parameter with latitude.
In the vorticity equation, this introduces a term proportional to the
northward velocity component $u_y$:
\begin{align}
  \beta u_y,
\end{align}
where $\beta$ is the linear expansion coefficient of the Coriolis
parameter.

Including this term in the vorticity transport equation gives:
\begin{align}
  \frac{\partial\omega}{\partial t} - J(\psi, \omega) + \beta u_y
  = \nu \nabla^2\omega - \mu\omega + \mathbf{f}_\omega.
\end{align}

The beta-plane term is crucial for studying large-scale atmospheric
and oceanic dynamics, as it leads to phenomena such as Rossby waves
and geostrophic turbulence.